# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/
https://edwarddonner.com/2025/01/23/ll

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/Qwen/Qwen3-235B-A22B',
 '/deepseek-ai/DeepSeek-Prover-V2-671B',
 '/nari-labs/Dia-1.6B',
 '/Qwen/Qwen3-30B-A3B',
 '/Qwen/Qwen3-32B',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/Qwen/Qwen3-Demo',
 '/spaces/nari-labs/Dia-1.6B',
 '/spaces/RiverZ/ICEdit',
 '/spaces/stepfun-ai/Step1X-Edit',
 '/spaces',
 '/datasets/nvidia/OpenMathReasoning',
 '/datasets/nvidia/Nemotron-CrossThink',
 '/datasets/nvidia/OpenCodeReasoning',
 '/datasets/rajpurkarlab/ReXGradient-160K',
 '/datasets/OpenGVLab/InternVL-Data',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',
 '/docs/diffusers',
 '/docs/safe

In [14]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'github page', 'url': 'https://github.com/huggingface'},
  {'type': 'linkedin page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'discuss forum', 'url': 'https://discuss.huggingface.co'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [15]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [16]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
Qwen/Qwen3-235B-A22B
Updated
5 days ago
•
30.5k
•
685
deepseek-ai/DeepSeek-Prover-V2-671B
Updated
5 days ago
•
2.03k
•
660
nari-labs/Dia-1.6B
Updated
9 days ago
•
113k
•
1.84k
Qwen/Q

In [17]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [18]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [19]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company blog', 'url': 'https://huggingface.co/blog'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'company profile on LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nQwen/Qwen3-235B-A22B\nUpdated\n5 days ago\n•\n30.5k\n•\n685\ndeepseek-ai/DeepSeek-Prover-V2-671B\nUpdated\n5 days ago\n•\n2.03k\n•\n660\nnari-labs/Dia-1.6B\nUpdated\n9 days ago\n•\n113k\n•\n1.84k\nQwen/Qwen3-30B-A3B\nUpdated\n6 days ago\n•\n67.6k\n•\n449\nQwen/Qwen3-32B\nUpdated\n6 days ago\n•\n87.4k\n•\n269\nBrowse 1M+ models\nSpaces\nRunning\n5.94k\n5.94k\nDeepSite\n🐳\nGenerate any applicati

In [20]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [27]:
create_brochure("WFMU", "https://wfmu.org")

Found links: {'links': [{'type': 'about page', 'url': 'https://wfmu.org/about/'}, {'type': 'donate page', 'url': 'https://wfmu.org/donate-stuff/'}, {'type': 'careers page', 'url': 'https://wfmu.org/employer-matching-donations/'}, {'type': 'event page', 'url': 'https://wfmu.org/recordfair/'}, {'type': 'blog', 'url': 'https://blog.wfmu.org/'}]}


# WFMU Brochure

---

## **About Us**
Welcome to **WFMU**, an independent freeform radio station broadcasting at 91.1 FM in New Jersey and 90.1 FM in the Hudson Valley. Since our inception, we have committed ourselves to a philosophy of freeform broadcasting, ensuring that each show is a unique listening experience that values creativity and artistic expression over commercial interests.

### **Our Mission**
At WFMU, we focus on human-based programming, actively steering clear of algorithms and mainstream trends. Our goal is to provide a platform for diverse voices and unconventional sounds, making every broadcast feel fresh and engaging.

## **Why Choose WFMU?**
- **Diverse Programming**: We offer a wide variety of shows, such as Rock ‘n’ Soul Radio and Sheena’s Jungle Room, along with a live stream that keeps the music flowing around the clock.
- **Vibrant Community**: Join a loyal listener base that values creativity and the free exchange of ideas. Our listeners are not just passive consumers but active participants in the station’s culture.
- **Extensive Online Presence**: In addition to our FM broadcasts, we provide "gobs and gobs of online offerings," including podcasts, playlists, live streams, and program archives to ensure you never miss a beat.

## **Company Culture**
WFMU thrives on a culture of inclusivity, creativity, and collaboration. We encourage artists, writers, and creators to engage with us through projects like our program guide/zine, "Lowest Common Denominator." Our intimate atmosphere fosters a sense of belonging, empowering both staff and artists to push the envelope in their creative pursuits.

### **Notable Events & Collaborations**
- **Live Performances**: We host live DJ sessions and unique events like "Radio Ravioli," bridging the gap between radio and live performance art.
- **Film Nights**: From audio-visual experiences to themed film nights, our Monty Hall venue is a hub for cultural engagement.

## **Listeners & Community Engagement**
Our diverse listeners range from passionate audiophiles to casual music lovers. We believe in cultivating a strong, engaged community that contributes to our programming. Whether it's through listener donations, submissions to our zine, or participation in our events, our audience plays a crucial role in defining the WFMU experience.

## **Careers at WFMU**
Join a dynamic team passionate about independent radio! WFMU values creativity and activism, and we seek individuals who share our love for music, arts, and community engagement. Opportunities for collaboration and performance are abundant, whether you're an experienced radio host, sound artist, or administrative professional.

### **Get Involved!**
- **Listen**: Tune in to our live streams or explore our extensive archive of shows.
- **Donate**: Support independent radio by contributing to our station.
- **Engage**: Join our community events, submit your creative works, or sign up for our monthly newsletter.

---

For more information about our programs, upcoming events, and how to get involved, please visit [WFMU.org](http://wfmu.org) or tune in to our broadcasts. 

**WFMU – Where Independent Voices Resonate!**

In [23]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [26]:
print(MODEL)

gpt-4o-mini


## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [24]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}



# Hugging Face Brochure

## Welcome to Hugging Face
**The AI community building the future.**  
At Hugging Face, we are dedicated to creating a collaborative platform for the machine learning community. Our goal is to empower users to create, discover, and share machine learning models, datasets, and applications seamlessly.

---

## What We Offer
- **Models:** Discover over 1 million machine learning models suited for various tasks.
- **Datasets:** Explore our extensive library of more than 250,000 datasets designed to enhance your AI projects.
- **Spaces:** Share and host your applications for unlimited collaboration.
- **Enterprise Solutions:** We support organizations with enterprise-grade security, dedicated resources, and priority support.

### Trending Models
- **Qwen/Qwen3-235B-A22B** – Latest update with a high engagement.
- **DeepSeek-Prover-V2-671B** – A popular model for advanced processing.

---

## Our Community 
Over **50,000 organizations** use Hugging Face, including industry leaders like:
- **Google**
- **Amazon**
- **Microsoft**
- **Meta**

Our community thrives on collaboration, sharing, and innovation, making it a dynamic environment for AI developers and enthusiasts.

---

## Company Culture
At Hugging Face, we believe in the power of open-source and collaboration. Our culture is built on:
- **Inclusivity:** We welcome contributors from all backgrounds.
- **Transparency:** Sharing knowledge and resources openly to foster learning.
- **Innovation:** Encouraging creative solutions and experimentation in AI development.

---

## Careers at Hugging Face
Join us in shaping the future of AI! We offer various opportunities for passionate individuals who are eager to contribute to the AI&ML ecosystem. Work alongside experts in a collaborative setting where your ideas are valued.

### Current Openings:
- Machine Learning Engineer
- Software Developer
- Community Manager

### Benefits:
- Competitive salary
- Flexibility in remote work
- Opportunities for professional growth and learning

---

## Get Started with Hugging Face
Ready to build the future of AI with us?  
**[Sign Up](#)** to start exploring our models, datasets, and applications or learn more about our **[Enterprise solutions](#)**.

Connect with us on **[Twitter](#), [GitHub](#),** and **[LinkedIn](#)**!

Together, let's innovate and push the boundaries of AI.



In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>